<a href="https://colab.research.google.com/github/minshyee/Competition_Project/blob/main/DACON/%EC%87%BC%ED%95%91%EB%AA%B0_%EB%A6%AC%EB%B7%B0_%ED%8F%89%EC%A0%90_%EB%B6%84%EB%A5%98_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Contest/dacon/shop_review_classification

!unzip -qq /content/drive/MyDrive/dataset.zip

/content/drive/MyDrive/Contest/dacon/shop_review_classification
replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace dataset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [6]:
!pwd

/content/drive/MyDrive/Contest/dacon/shop_review_classification


In [7]:
import os
import pandas as pd
import numpy as np

In [11]:
data_src = '/content/drive/MyDrive/Contest/dacon/shop_review_classification/dataset'

train_src = os.path.join(data_src, 'train.csv')
df = pd.read_csv(train_src, index_col='id') # 확인 결과 index와 id 동일 개수

In [16]:
df.head()

,reviews,target
id,,
0,조아요 처음구입 싸게햇어요,2
1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,튼튼하고 손목을 잘 받쳐주네요~,5


In [17]:
df.tail()

,reviews,target
id,,
24995,일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여,5
24996,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.,2
24997,향이 좋아요!! 다른 향도 시켜보고 싶어요,4
24998,펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다,2
24999,호호애미 제품은 늘 믿고 써요,5


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  25000 non-null  object
 1   target   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 585.9+ KB


- id | reviews | target
- 3*25000
- 결측치 없음

In [22]:
# target 분포
df.target.value_counts()

5    10000
2     8000
1     4500
4     2500
Name: target, dtype: int64

# BASELINE 따라해보기

## Text Preprocessing
- CountVectorizer
토큰의 등장 빈도 벡터로 

In [24]:
from sklearn.feature_extraction.text import CountVectorizer 

In [28]:
c_vectorizer = CountVectorizer()
c_vectorizer.fit(df.reviews)

CountVectorizer()

## train

In [35]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(df, test_size=0.25, stratify = df.target)

In [39]:
train_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)

In [40]:
print('학습시킬 train 셋 : ', train_data.shape)
print('검증할 val 셋 : ', val_data.shape)

학습시킬 train 셋 :  (18750, 3)
검증할 val 셋 :  (6250, 3)


In [41]:
# 변수 설정
train_X = train_data.reviews
train_y = train_data.target

val_X = val_data.reviews
val_y = val_data.target

In [42]:
train_X = c_vectorizer.transform(train_X)
val_X = c_vectorizer.transform(val_X)

In [43]:
print(train_X[10])

  (0, 9656)	1
  (0, 10440)	1
  (0, 44309)	1


In [45]:
c_vectorizer.inverse_transform(train_X[10])

[array(['근데', '기우네요', '아주만족합니다'], dtype='<U100')]

In [46]:
from sklearn.linear_model import LogisticRegression # 모델 불러오기
model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(train_X, train_y) #모델 학습

LogisticRegression(max_iter=500)

In [47]:
# 먼저 점수를 메기는 방법인 평가 지표(Metric)를 정의합니다.
import numpy as np

def ACCURACY(true, pred):   
    score = np.mean(true==pred)
    return score

In [48]:
# val_X를 이용해 예측값을 생성합니다.
y_pred = model.predict(val_X)

In [49]:
# 모델의 예측과 실제 정답값을 비교합니다.
score = ACCURACY(val_y, y_pred)

print(f"모델의 정확도는 {score*100:.2f}% 입니다")

모델의 정확도는 60.38% 입니다
